# PoC: WeCookio multiagent Strands Agents

This notebook demonstrates the WeCookio CrewAI implementation with cost-optimized model selection using uv package manager.

### Process.Sequential without delegation

### Installing Required Packages with uv

In [ ]:
# Install required packages using uv
!uv add ipykernel strands-agents==1.2.0 strands-agents-tools boto3 botocore pyyaml ipywidgets prompt-template

### Import Libraries and Load Configuration

In [ ]:
import yaml
import logging
from strands import Agent
from strands.multiagent import Swarm

print("Libraries imported successfully!")

# Enable debug logs and print them to stderr
logging.getLogger("strands.multiagent").setLevel(logging.DEBUG)
logging.basicConfig(
    format="%(levelname)s | %(name)s | %(message)s",
    handlers=[logging.StreamHandler()]
)


### Loading AWS Enviroment variables

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()
AWS_ACCESS_KEY_ID=os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY=os.getenv("AWS_SECRET_ACCESS_KEY")
AWS_DEFAULT_REGION=os.getenv("AWS_REGION")

### Load Configuration Files

In [ ]:
#Check base folder for relative paths
import os
print (os.getcwd())
prefix = "strands/latest/"
# Load the integration configuration
with open(prefix+'config/integration_config.yaml', 'r') as file:
    integration_config = yaml.safe_load(file)
# Load the crew configuration
with open(prefix+'config/config.yaml', 'r') as file:
    strand_config = yaml.safe_load(file)
with open(prefix+'config/agents-sequential.yaml', 'r') as file:
    agents_config = yaml.safe_load(file)
with open(prefix+'config/tasks-sequential.yaml', 'r') as file:
    tasks_config = yaml.safe_load(file)

print("Configuration files loaded successfully!")
print(f"Integration config keys: {list(integration_config.keys())}")
print(f"Crew Orchestration keys: {list(strand_config.keys())}")
print(f"Agents config keys: {list(agents_config.keys())}")
print(f"Tasks config keys: {list(tasks_config.keys())}")

###  Model Selection based on complexity

In [ ]:
def select_model_for_complexity(complexity, integration_config):
    """Select the appropriate model based on task complexity"""
    try:
        complexity = complexity + "_model"
        return integration_config["aws_bedrock"][complexity]
    except KeyError:
        raise ValueError(f"Invalid complexity level: {complexity}")

# Example usage
high_complexity_model = select_model_for_complexity("high_complexity", integration_config)
medium_complexity_model = select_model_for_complexity("medium_complexity", integration_config)
low_complexity_model = select_model_for_complexity("low_complexity", integration_config)
ultra_low_complexity_model = select_model_for_complexity("ultra_low_complexity", integration_config)

print(f"High complexity model: {high_complexity_model['model_id']}")
print(f"Medium complexity model: {medium_complexity_model['model_id']}")
print(f"Low complexity model: {low_complexity_model['model_id']}")
print(f"Ultra-Low complexity model: {ultra_low_complexity_model['model_id']}")

### Create Agents with Model-Specific Configuration

In [ ]:

from datetime import datetime
import boto3
from strands.models import BedrockModel
from prompt_template import PromptTemplate, InvalidTemplateKeysError, MissingTemplateValuesError, TemplateSerializationError

# Global metrics storage
execution_metrics = {
    'start_time': None,
    'total_tokens': 0,
    'total_cost': 0.0,
    'cycle_durations': [],
    'tool_usage': {},
    'errors': [],
    'agent_performance': {}
}

def safe_get_metrics(kwargs):
    """Safely extract metrics from kwargs"""
    try:
        metrics = kwargs.get('metrics')
        if metrics is None:
            return {
                'accumulated_usage': {'totalTokens': 0},
                'cycle_durations': [],
                'tool_metrics': {}
            }
        return metrics
    except Exception as e:
        print(f"⚠️  Warning: Could not extract metrics: {e}")
        return {
            'accumulated_usage': {'totalTokens': 0},
            'cycle_durations': [],
            'tool_metrics': {}
        }

def event_loop_tracker(**kwargs):
    """Enhanced event loop tracker with proper error handling and metrics collection"""
    try:
        # Track event loop lifecycle
        if kwargs.get("init_event_loop", False):
            print("🔄 Event loop initialized")
            execution_metrics['start_time'] = datetime.now()
        elif kwargs.get("start_event_loop", False):
            print("▶️ Event loop cycle starting")
        elif kwargs.get("start", False):
            print("📝 New cycle started")
        elif "message" in kwargs:
            print(f"📬 New message created: {kwargs['message']['role']}")
        elif kwargs.get("complete", False):
            print("✅ Cycle completed")
            # Update global metrics
            metrics = safe_get_metrics(kwargs)
            if metrics['cycle_durations']:
                execution_metrics['cycle_durations'].extend(metrics['cycle_durations'])
            if metrics['accumulated_usage']['totalTokens']:
                execution_metrics['total_tokens'] += metrics['accumulated_usage']['totalTokens']
        elif kwargs.get("force_stop", False):
            print(f"🛑 Event loop force-stopped: {kwargs.get('force_stop_reason', 'unknown reason')}")
            execution_metrics['errors'].append({
                'type': 'force_stop',
                'reason': kwargs.get('force_stop_reason', 'unknown'),
                'timestamp': datetime.now().isoformat()
            })

        # Track tool usage
        if "current_tool_use" in kwargs and kwargs["current_tool_use"].get("name"):
            tool_name = kwargs["current_tool_use"]["name"]
            print(f"🔧 Using tool: {tool_name}")
            execution_metrics['tool_usage'][tool_name] = execution_metrics['tool_usage'].get(tool_name, 0) + 1

        # Show metrics when data is available
        #if "data" in kwargs:
            # Only show first 20 chars of each chunk for demo purposes
            #data_snippet = kwargs["data"][:20] + ("..." if len(kwargs["data"]) > 20 else "")
            #print(f"📟 Text: {data_snippet}")   
    except Exception as e:
        error_msg = f"Error in event loop tracker: {e}"
        print(f"❌ {error_msg}")
        execution_metrics['errors'].append({
            'type': 'event_loop_tracker_error',
            'message': str(e),
            'timestamp': datetime.now().isoformat()
        })
    
def agent_description_composer(agent_config):
    """Compose the agent description"""
    return f"""
    Role: {agent_config.get("role", None)}
    Goal: {agent_config.get("goal", None)}
    Backstory: {agent_config.get("backstory", None)}
    """

def create_agent_with_model(agent_config, model_config, **kwargs):
    """Create an agent with specific model configuration"""
    llm = BedrockModel(
        model_id=model_config["model_id"].split("/")[-1],
        temperature=model_config.get("model_kwargs").get("temperature", 0.6),
        top_p=model_config.get("model_kwargs").get("top_p", 0.9),
        max_tokens=model_config.get("model_kwargs").get("max_tokens", 4000)
    )

    args = {}
    args["model"]=llm
    args["system_prompt"]=agent_config.get("prompt", None)
    args["name"]=agent_config.get("name", None)
    args["description"]=agent_description_composer(agent_config)
    args["callback_handler"]=event_loop_tracker
    
    if not agent_config.get("verbose",True):
        args["callback_handler"]=None
    
    return Agent(**args)

def print_agent_info(agent, model_config) -> None:
    """Print agent creation information."""
    print(
        f"Created agent: {agent.name}\n"
        f"Agent Description: {agent.description}\n"
        f"Using model: {agent.model.get_config()}"
    )
    print("\n")

In [ ]:

#agent_name = "Culinary Coordinator"
agent_config = agents_config["Culinary Coordinator"]
agent_type=agent_config["name"]
model_config = select_model_for_complexity(strand_config["model_assignment"]["agent_model_mapping"][agent_type], integration_config)
culinary_coordinator_agent = create_agent_with_model(agent_config, model_config)
print_agent_info(culinary_coordinator_agent, model_config)

agent_name = "Ingredient Substitution Expert"
agent_config = agents_config["Ingredient Substitution Expert"]
agent_type=agent_config["name"]
model_config = select_model_for_complexity(strand_config["model_assignment"]["agent_model_mapping"][agent_type], integration_config)
ingredient_substitution_expert_agent = create_agent_with_model(agent_config, model_config)
print_agent_info(ingredient_substitution_expert_agent, model_config)

agent_name = "Culinary Experience Optimizer"
agent_config = agents_config["Culinary Experience Optimizer"]
agent_type=agent_config["name"]
model_config = select_model_for_complexity(strand_config["model_assignment"]["agent_model_mapping"][agent_type], integration_config)
culinary_experience_optimizer_agent = create_agent_with_model(agent_config, model_config)
print_agent_info(culinary_experience_optimizer_agent, model_config)

agent_name = "Quality Assurance Chef"
agent_config = agents_config["Quality Assurance Chef"]
agent_type=agent_config["name"]
model_config = select_model_for_complexity(strand_config["model_assignment"]["agent_model_mapping"][agent_type], integration_config)
quality_assurance_chef_agent = create_agent_with_model(agent_config, model_config)
print_agent_info(quality_assurance_chef_agent, model_config)


### Create a Flow

In [ ]:
# Example recipe for analysis
recipe_request = {
    "food_name": "Canelones de ternera XXL al Pedro Ximenez",
    "servings": 4,
    "intolerances": [
        "Milk",
        "Eggs",
        "Beef",
        "Corn",
        "Gluten",
        "Rice",
        "Garlic"
    ],
    "exclusions": ["Pepper"],
    "preferences": ["kosher"],
    "output_language": "spanish",
    "country": "Spain"
}
initial_recipe={}
initial_recipe["food_name"]=recipe_request.pop("food_name")
initial_recipe["country"]=recipe_request["country"]
initial_recipe["output_language"]=recipe_request["output_language"]
initial_recipe["servings"]=recipe_request["servings"]


In [ ]:
## Support functions

def variable_injection(task:dict, recipe_context:dict):
    try:
        prompt = PromptTemplate(name=task.get("name"), template=f'''{task.get("description")}''').to_string(**recipe_context)
    except MissingTemplateValuesError as e:
        print(f"Missing values: {e.missing_values}")  # {'name'}
    except InvalidTemplateKeysError as e:
        print(f"Invalid keys: {e.invalid_keys}")  # {'name'}
    except TemplateSerializationError as e:
        print(f"Serialization error: {e.message}")  # 'Invalid template string'
    except Exception as e:
        print(f"Unexpected error: {e}")  # 'Unexpected error'
    return prompt
    

In [ ]:
# Sequential workflow processing
tasks = {
    "initial_recipe_analysis": {
        "description": f'''{variable_injection(tasks_config['Initial Recipe Analysis'], initial_recipe)}''',
        "status": "pending",
        "agent": culinary_coordinator_agent,
        "dependencies": []
    },
    "ingredient_substitution_strategy": {
        "description": f'''{variable_injection(tasks_config['Ingredient Substitution Strategy'], recipe_request)}''',
        "status": "pending",
        "agent": ingredient_substitution_expert_agent,
        "dependencies": ["initial_recipe_analysis"]
    },
    "culinary_experience_enhancement": {
        "description": f'''{variable_injection(tasks_config['Culinary Experience Enhancement'], recipe_request)}''',
        "status": "pending",
        "agent": culinary_experience_optimizer_agent,
        "dependencies": ["ingredient_substitution_strategy"]
    },
    "final_quality_assurance": {
        "description": f'''{variable_injection(tasks_config['Final Quality Assurance'], recipe_request)}''',
        "status": "pending",
        "agent": quality_assurance_chef_agent,
        "dependencies": ["culinary_experience_enhancement"]
    }
}


In [ ]:

# Sequential workflow processing
def weCookio_process_workflow(tasks:dict):
    print(f''' Flow:\n {tasks} \n ''')
    context = ''
    for task in tasks:
        if tasks[task]['dependencies']:
            print(f'''\nInjecting dependency: {tasks[task]['dependencies']}''')
            task_result = tasks[task]['agent'](f'''Previous task results:{context}\n\n Task: \n {tasks[task]['description']}''')
        else:
            print(f'''Injecting task: {tasks[task]['description']}\n''')
            task_result = tasks[task]['agent'](f'''Task: \n {tasks[task]['description']}''')
        tasks[task]['result'] = task_result
        context = task_result
        print (context)
    return tasks        
    

### Run the Flow

In [ ]:
try:
    result = weCookio_process_workflow(tasks=tasks)
except Exception as e:
    print(f"Error running Workflow: {e}")
    print("Make sure you have AWS credentials configured and the required permissions.")


In [ ]:
from pprint import pprint
#pprint("".join(result['initial_recipe_analysis']['result'].message['content'][0]['text']))
print("".join(result['final_quality_assurance']['result'].message['content'][0]['text']))
print(f'''\n\n Aggregated Metrics: \n\n''')
pprint(result['final_quality_assurance']['result'].metrics)



